In [2]:
import os

os.add_dll_directory(r"C:\OpenCV\opencv-4.10.0\build\install\x64\vc16\bin")
os.add_dll_directory(r"C:\OpenCV\opencv-4.10.0\build\bin\Release")

<AddedDllDirectory('C:\\OpenCV\\opencv-4.10.0\\build\\bin\\Release')>

In [3]:
import cv2 as cv
import numpy as np
import argparse
import networkx as nx

from os import listdir

In [4]:
def matchWithRatioTest(
    matcher: cv.DescriptorMatcher, desc1, desc2, nn_match_ratio_th=0.8
):
    nn_matches = matcher.knnMatch(desc1, desc2, 2)
    ratioMatched = []
    for m, n in nn_matches:
        if m.distance < n.distance * nn_match_ratio_th:
            ratioMatched.append(m)

    return ratioMatched

In [5]:
"""
Clase Imagen

Almacena el nombre de la imagen, keypoints, descriptores y un diccionario con los emparejamientos que tiene.
"""


class Image:
    def __init__(self, img_name, image, keypoints, descriptors):
        self.img_name = img_name
        self.image = image
        self.keypoints = keypoints
        self.descriptors = descriptors
        self.matches = {}

    def __hash__(self):
        return hash(self.img_name)

    def __str__(self):
        return f"Imagen: {self.img_name} \n # Keypoints: {len(self.keypoints)} \n # Descriptores: {self.descriptors.shape} \n Matches: {list(self.matches.keys())} \n"


"""
Clase ImageFeature
Almacena la referencia a una imagen (nombre del archivo) junto con un índice del keypoint específico a esa imagen
"""


class ImageFeature:
    def __init__(self, img_name, kpt_idx):
        self.img_name = img_name
        self.kpt_idx = kpt_idx

    def __hash__(self):
        return hash(self.img_name + str(self.kpt_idx))

    def __str__(self):
        return f"{self.img_name}, keypoint: {self.kpt_idx}"

In [6]:
PAIR_MATCH_SURVIVAL_RATE = 0.5

In [7]:
# Diccionario de imágenes
images = {}

# Carpeta de las imagenes
images_folder = "./images"

# Leer todos los archivos dentro de la carpeta
images_files = listdir(images_folder)

# Inicializar descriptor AKAZE
akaze = cv.AKAZE_create()
# Almacenamos todos los keypoints y descriptors que encontramos
keypoints = []
descriptors = []
for img in images_files:
    img_g = cv.imread(images_folder + "/" + img, cv.IMREAD_GRAYSCALE)
    if img_g is None:
        print("No se pudo abrir la imagen " + img)
        exit(0)

    # Inicializar el descriptor AKAZE y detectar keypoints y extraer los descriptores de la imagen.
    # kpts -> keypoints
    # descs -> descriptores
    kpts, descs = akaze.detectAndCompute(img_g, None)
    print("keypoints: {}, descriptors: {}".format(len(kpts), descs.shape))

    # Crear la clase imagen con sus keypoints y descriptors
    # Se incluye en el diccionario de imágenes
    images[img] = Image(img, img_g, kpts, descs)

    # Mostrar resultados de la extracción
    # showKeypoints(img_g, kpts)

keypoints: 1837, descriptors: (1837, 61)
keypoints: 2171, descriptors: (2171, 61)
keypoints: 2166, descriptors: (2166, 61)
keypoints: 1996, descriptors: (1996, 61)


In [8]:
# Fase de Matching

# Crear matches para todas las imágenes
# Utilizamos distancia de Hamming porque AKAZE es un descriptor binario
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_BRUTEFORCE_HAMMING)

for i, img_i in enumerate(list(images.values())):
    for j, img_j in enumerate(list(images.values())[i + 1 : :]):
        # Encontrar los match
        matches = matchWithRatioTest(
            matcher, img_i.descriptors, img_j.descriptors
        )

        # Test de reciprocidad
        matchRcp = matchWithRatioTest(
            matcher, img_j.descriptors, img_i.descriptors
        )

        merged_matches = []

        for mR_m in matchRcp:
            found = False
            for m_m in matches:
                # Solo se acepta el match si es recíproco
                if (
                    mR_m.queryIdx == m_m.trainIdx
                    and mR_m.trainIdx == m_m.queryIdx
                ):
                    merged_matches.append(m_m)
                    found = True
                    break
                if found:
                    continue

        # Restricción Epipolar. Calculamos la matriz fundamental utilizando RANSAC
        img_i_pts = []
        img_j_pts = []

        for m in merged_matches:
            img_i_pts.append(img_i.keypoints[m.queryIdx].pt)
            img_j_pts.append(img_j.keypoints[m.trainIdx].pt)

        F, inliersMask = cv.findFundamentalMat(
            np.array([img_i_pts]), np.array([img_j_pts]), cv.FM_RANSAC
        )

        final = []

        for mask, match in zip(inliersMask, merged_matches):
            if mask:
                final.append(match)

        if float(len(final)) / float(len(matches)) < PAIR_MATCH_SURVIVAL_RATE:
            print(
                "El match final tiene menos de la mitad de matches que el original. Ignorando imagen."
            )
            continue

        images[img_i.img_name].matches[
            (img_i.img_name, img_j.img_name)
        ] = final

El match final tiene menos de la mitad de matches que el original. Ignorando imagen.
El match final tiene menos de la mitad de matches que el original. Ignorando imagen.
El match final tiene menos de la mitad de matches que el original. Ignorando imagen.


In [9]:
# Crear grafo de características para rastrearlas

vertexByImageFeature = {}

G = nx.Graph()

# Agregamos nodos con las imágenes
for img_tpl in images.values():
    for i in range(len(img_tpl.keypoints)):
        node = ImageFeature(img_tpl.img_name, i)
        G.add_node(node)
        vertexByImageFeature[(img_tpl.img_name, i)] = node

# Agregamos las aristas (matches)
for img_tpl in images.values():
    for matches in img_tpl.matches:
        for m in img_tpl.matches[matches]:
            v1 = vertexByImageFeature[(matches[0], m.queryIdx)]
            v2 = vertexByImageFeature[(matches[1], m.trainIdx)]
            G.add_edge(v1, v2)

# Eliminar nodos sin aristas
G.remove_nodes_from(list(nx.isolates(G)))

# Obtener los componentes conectados (retorna una lista de sets con nodos (componentes))
components = nx.connected_components(G)
goodComponents = list()
# Filtramos los componentes (más de 1 característica por imagen)
for c in components:
    isGoodComponent = True
    imagesInComponent = set()
    for img in c:
        img_name = img.img_name
        if img_name in imagesInComponent:
            # La imagen ya se encuentra en el componente
            isGoodComponent = False
            break
        else:
            imagesInComponent.add(img_name)
    if isGoodComponent:
        goodComponents.append(c)

print(f"Total de componentes: {nx.number_connected_components(G)}")
print(f"Componentes buenos: {len(goodComponents)}")

Total de componentes: 642
Componentes buenos: 642


In [10]:
# Preparación de Tracks para alimentar sfm
tracks = {}

# Creamos un array de tracks inicializados en -1
for img in images_files:
    tracks[img] = np.full((2, len(goodComponents)), -1.0, np.float64)


for (i,c) in enumerate(goodComponents):
    for img in c:
        img_name = img.img_name
        kpt_idx = img.kpt_idx
        p = images[img_name].keypoints[kpt_idx].pt
        tracks[img_name][0][i] = p[0]
        tracks[img_name][1][i] = p[1]

tracks_list = np.fromiter(tracks.values(), dtype=object)

In [11]:
tracks_list[0]

array([[ 3.49071631e+03,  3.12157007e+03,  3.18299341e+03, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00],
       [ 1.39102271e+03,  1.42428687e+03,  1.59039038e+03, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]])

In [12]:
help(cv.sfm.reconstruct)

Help on built-in function reconstruct:

reconstruct(...)
    reconstruct(points2d, K[, Rs[, Ts[, points3d[, is_projective]]]]) -> Rs, Ts, K, points3d
    .   @brief Reconstruct 3d points from 2d correspondences while performing autocalibration.
    .     @param points2d Input vector of vectors of 2d points (the inner vector is per image).
    .     @param Rs Output vector of 3x3 rotations of the camera.
    .     @param Ts Output vector of 3x1 translations of the camera.
    .     @param points3d Output array with estimated 3d points.
    .     @param K Input/Output camera matrix \f$K = \vecthreethree{f_x}{0}{c_x}{0}{f_y}{c_y}{0}{0}{1}\f$. Input parameters used as initial guess.
    .     @param is_projective if true, the cameras are supposed to be projective.
    .
    .     Internally calls libmv simple pipeline routine with some default parameters by instatiating SFMLibmvEuclideanReconstruction class.
    .
    .     @note
    .       - Tracks must be as precise as possible. It does

In [13]:
imgS = images[list(images.keys())[0]].image.shape

f = np.float64(np.max(imgS))


# Camera Matrix
# TODO: Incluir la calibración de la cámara en este NB
K = np.array([[f, 0.0, imgS[1]/2.0], [0.0, f, imgS[0] / 2.0], [0.0, 0.0, 1.0]])

In [14]:
Rs, Ts, K, points3d = cv.sfm.reconstruct(points2d=tracks_list, K=K, is_projective=True)

In [15]:
print("Reconstrucción:")
print("Puntos 3D estimados: "+str(len(points3d)))
print("Rotaciones estimadas de camara: " + str(len(Rs)))
print("Traslaciones estimadas de camara: " + str(len(Ts)))
print("Parametros intrinsecos refinados: ")
print(K)

Reconstrucción:
Puntos 3D estimados: 642
Rotaciones estimadas de camara: 4
Traslaciones estimadas de camara: 4
Parametros intrinsecos refinados: 
[[3.93311136e+03 0.00000000e+00 2.68810129e+03]
 [0.00000000e+00 3.93311136e+03 1.57017989e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [16]:
# Convertir a un array de vectores de 3x1 (Puntos 3D)
pointCloud = np.array(points3d)
pointCloud.shape


(642, 3, 1)

In [17]:
# Obtener los colores de los puntos
defaultColor = np.array([0,255,0])
pointCloudColor = np.tile(defaultColor, (pointCloud.shape[0], 1))
pointCloudColor.shape

(642, 3)

In [18]:
# Buscar el color del punto en todas las imágenes
for i,point in enumerate(pointCloud):
    for img, R, T in zip(images.values(), Rs, Ts):
        point3d = np.reshape(point, (3,))
        point2d = cv.projectPoints(point3d, R, T, K, None)
        # Si el punto se encuentra fuera de los límites
        x = int(point2d[0][0][0][0])
        y = int(point2d[0][0][0][1])
       
        if (
            x < 0
            or x >= img.image.shape[0]
            or y < 0
            or y >= img.image.shape[1]
        ):
            continue

        pointCloudColor[i] = np.array(img.image[x, y])
        break


pointCloudColor.shape

(642, 3)

In [24]:
# Visualizar en ventana 3D
window = cv.viz.Viz3d("Frame de Coordenadas")
window.setWindowSize((500,500))
window.setWindowPosition((150,150))
window.setBackgroundColor(cv.viz.Color.white())

# Recuperar camaras


In [39]:
Affine = np.concatenate((Rs[0], Ts[0]), axis = 1)
Affine = np.concatenate((Affine, np.array([[0,0,0,1]])))
Affine


array([[ 0.93502787,  0.03895333, -0.35242803,  1.21109839],
       [-0.03841536,  0.99922551,  0.00852295, -0.04044635],
       [ 0.35248707,  0.00556945,  0.93580011,  0.22705069],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])